# Week 13: Model Deployment
## Course: Applied Data Science with AI
## Project: House Price Prediction

This notebook demonstrates how to deploy a trained machine learning model
using Flask as a simple API running on localhost.


#### Imports

In [2]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


#### Load Dataset

In [3]:
df = pd.read_csv(r"C:\Users\123\Documents\Semester 7\Data-Science-AI-Course\Data\Raw\house_prices.csv")

df.columns = df.columns.str.strip()
print("Columns:", df.columns.tolist())


Columns: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',

#### Features & Target

In [4]:
target_column = 'SalePrice'

X = df.select_dtypes(include=[np.number]).drop(target_column, axis=1)
y = df[target_column]

X = X.fillna(0)
y = y.fillna(y.mean())


#### Train Model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("Model trained successfully!")


Model trained successfully!


In [8]:
import pickle

# Save model
pickle.dump(model, open("house_price_model.pkl", "wb"))

# Save scaler
pickle.dump(scaler, open("scaler.pkl", "wb"))


#### Save Model & Scaler (For Deployment)

In [7]:
with open("house_price_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and scaler saved!")


Model and scaler saved!


#### Flask API Code (Deployment Cell)

⚠️ This cell is for code writing, not execution inside Jupyter.

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
import pickle

app = Flask(__name__)

model = pickle.load(open("house_price_model.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))

@app.route("/")
def home():
    return "House Price Prediction API is running!"

@app.route("/predict", methods=["POST"])
def predict():
    data = request.json
    features = np.array(data["features"]).reshape(1, -1)
    features_scaled = scaler.transform(features)
    prediction = model.predict(features_scaled)
    return jsonify({"Predicted Price": float(prediction[0])})

if __name__ == "__main__":
    app.run(debug=True)


### How to Run the API

1. Save the Flask code in a file named `app.py`
2. Open Anaconda Prompt
3. Navigate to the folder
4. Run:

```bash
python app.py



---

## 9️⃣ Test API (Example)

```markdown
POST Request Example (JSON):

{
  "features": [60, 8450, 7, 5, 2003, 2003, 856, 854, 1710, 2, 1, 3, 8, 0, 2, 548, 0]
}
